In [2]:
# library
import matplotlib.pyplot as plt
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import pickle

from io import StringIO

import urllib

from io import BytesIO

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sambe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sambe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Open spreadsheets to be added to matrix

In [4]:
to_add1 = pd.read_csv(r"C:\Users\sambe\Projects\rookieplay\data\compile\Job Descriptions 1.csv")
to_add2 = pd.read_csv(r"C:\Users\sambe\Projects\rookieplay\data\compile\Job Descriptions 2.csv")
to_add3 = pd.read_csv(r"C:\Users\sambe\Projects\rookieplay\data\compile\Job Descriptions 3.csv")

In [8]:
## Examine all new rows
pd.set_option('display.max_rows', 100)

## Open existing spreadsheet and compare

In [13]:
job_descriptions = pd.read_csv('pkl/job_descriptions.csv', index_col=0)
job_descriptions

,title,category,description,location,rake_key_words,bag_of_words
0,training_manager,management,The world's leading private security organizat...,NY,"['provide', 'limited', 'express', 'interest', ...",provide limited express interest additional tr...
1,training_manager,management,The Training Manager is responsible for the de...,NY,"['workshops', 'including', 'schedules', 'integ...",workshops including schedules integrity respon...
2,training_manager,management,The Training Manager of Applied Data Analytics...,NY,"['oversee', 'translate', 'diverse', 'audience'...",oversee translate diverse audience advanced kn...
3,training_manager,management,Unqork is the no-code platform that's pioneeri...,NY,"['public', 'speaking', '•', 'enjoy', 'getting'...",public speaking • enjoy getting previous exper...
4,training_manager,management,The Training Manager is in charge of training ...,NY,"['luxury', 'business', 'pr', 'teams', 'powerpo...",luxury business pr teams powerpoint paced envi...
...,...,...,...,...,...,...
260,dietitian (pool),"sports, fitness, & recreation",The Dietitian plans therapeutic diets and conf...,"phenix city, AL","['productivity', 'standards', 'equipment', 'co...",productivity standards equipment compliance ni...
261,youth services aide,"sports, fitness, & recreation",The Youth Services Aide is a permanent full-ti...,"montgomery, AL","['time', 'position', 'placed', 'may', 'apply',...",time position placed may apply directly locate...
262,nutritionist associate,"sports, fitness, & recreation","The Nutritionist Associate is a permanent, ful...","montgomery, AL","['time', 'position', 'local', 'health', 'agenc...",time position local health agency professional...
263,Licensed Real Estate SalesAgent,real estate,Job Description The Licensed Real Estate Sales...,"Spanish Fort, AL","['integrity', 'career', 'online', 'commission'...",integrity career online commission region pers...


In [15]:
all_to_add = [to_add1, to_add2, to_add3]

result = pd.concat(all_to_add)

In [22]:
result = result.reset_index(drop=True)

In [23]:
result

,JOB TITLE,JOB DESCRIPTION,JOB CATEGORY,LOCATION
0,Office Clerk,"As a Post Office Clerk, you are responsible fo...",Admin & Office,US
1,Inside Sales Representative,This is an inside sales position.\nThis is our...,Sales & Retail,"Fort Worth, TX"
2,Entry Level Collections Specialist,Why you should be interested in this role!\nTi...,Accounting & Finance,"Bothell, WA 98021"
3,Foreign Language Instructor,"As a Foreign Language Instructor for the CIA, ...",Education,"Washington, DC"
4,QA Tester,Quality Assurance Testers (“QA Testers”) perfo...,Computer & IT,"Baton Rouge, LA 70820"
...,...,...,...,...
295,Scor3card Coordinator,The Scor3card Coordinator is will report direc...,Social Services & Nonprofit,"Tulsa, OK"
296,Data Entry Specialist,Power Sales Group is a manufacturer's represen...,Admin & Office,"Danvers, MA"
297,Pathway Advisor,This is an exciting time to be an advisor at t...,Education,"Aurora, CO"
298,Director Of Food And Beverage,Seeking an experienced and dynamic Director of...,Management,"Sedona, AZ"


In [29]:
result = result.rename(columns = {"JOB TITLE": "title", "JOB DESCRIPTION": "description", "JOB CATEGORY": "category", "LOCATION": "location"})

In [30]:
result

,title,description,category,location
0,Office Clerk,"As a Post Office Clerk, you are responsible fo...",Admin & Office,US
1,Inside Sales Representative,This is an inside sales position.\nThis is our...,Sales & Retail,"Fort Worth, TX"
2,Entry Level Collections Specialist,Why you should be interested in this role!\nTi...,Accounting & Finance,"Bothell, WA 98021"
3,Foreign Language Instructor,"As a Foreign Language Instructor for the CIA, ...",Education,"Washington, DC"
4,QA Tester,Quality Assurance Testers (“QA Testers”) perfo...,Computer & IT,"Baton Rouge, LA 70820"
...,...,...,...,...
295,Scor3card Coordinator,The Scor3card Coordinator is will report direc...,Social Services & Nonprofit,"Tulsa, OK"
296,Data Entry Specialist,Power Sales Group is a manufacturer's represen...,Admin & Office,"Danvers, MA"
297,Pathway Advisor,This is an exciting time to be an advisor at t...,Education,"Aurora, CO"
298,Director Of Food And Beverage,Seeking an experienced and dynamic Director of...,Management,"Sedona, AZ"


In [38]:
cols = result.columns.tolist()

In [39]:
cols

['title', 'description', 'category', 'location']

In [41]:
cols = [cols[0], cols[2], cols[1], cols[3]]

In [42]:
cols

['title', 'category', 'description', 'location']

In [44]:
result = result[cols]

In [63]:
result

,title,category,description,location
0,Office Clerk,Admin & Office,"As a Post Office Clerk, you are responsible fo...",US
1,Inside Sales Representative,Sales & Retail,This is an inside sales position.\nThis is our...,"Fort Worth, TX"
2,Entry Level Collections Specialist,Accounting & Finance,Why you should be interested in this role!\nTi...,"Bothell, WA 98021"
3,Foreign Language Instructor,Education,"As a Foreign Language Instructor for the CIA, ...","Washington, DC"
4,QA Tester,Computer & IT,Quality Assurance Testers (“QA Testers”) perfo...,"Baton Rouge, LA 70820"
...,...,...,...,...
295,Scor3card Coordinator,Social Services & Nonprofit,The Scor3card Coordinator is will report direc...,"Tulsa, OK"
296,Data Entry Specialist,Admin & Office,Power Sales Group is a manufacturer's represen...,"Danvers, MA"
297,Pathway Advisor,Education,This is an exciting time to be an advisor at t...,"Aurora, CO"
298,Director Of Food And Beverage,Management,Seeking an experienced and dynamic Director of...,"Sedona, AZ"


In [46]:
## Adds document text to existing dataframe of job descriptions, extracting just the titles and text
def compile_document_text(text):
    with open('recs/pkl/job_descriptions.pkl', 'rb') as f:
        job_descriptions = pickle.load(f)
    data = [['resume', text]]
    basic_documentdf = pd.DataFrame(data, columns = ['title', 'description'])
    return basic_documentdf

In [58]:
result.columns

Index(['title', 'category', 'description', 'location'], dtype='object')

In [65]:
data = result[['title', 'description']]

In [69]:
result

,title,category,description,location
0,Office Clerk,Admin & Office,"As a Post Office Clerk, you are responsible fo...",US
1,Inside Sales Representative,Sales & Retail,This is an inside sales position.\nThis is our...,"Fort Worth, TX"
2,Entry Level Collections Specialist,Accounting & Finance,Why you should be interested in this role!\nTi...,"Bothell, WA 98021"
3,Foreign Language Instructor,Education,"As a Foreign Language Instructor for the CIA, ...","Washington, DC"
4,QA Tester,Computer & IT,Quality Assurance Testers (“QA Testers”) perfo...,"Baton Rouge, LA 70820"
...,...,...,...,...
295,Scor3card Coordinator,Social Services & Nonprofit,The Scor3card Coordinator is will report direc...,"Tulsa, OK"
296,Data Entry Specialist,Admin & Office,Power Sales Group is a manufacturer's represen...,"Danvers, MA"
297,Pathway Advisor,Education,This is an exciting time to be an advisor at t...,"Aurora, CO"
298,Director Of Food And Beverage,Management,Seeking an experienced and dynamic Director of...,"Sedona, AZ"


In [67]:
## Find key words from text using RAKE and transform those keywords into a bag of words
def text_to_bagofwords(df):
    df['rake_key_words'] = ''
    r = Rake()
    for index, row in df.iterrows():
        r.extract_keywords_from_text(row['description'])
        key_words_dict_scores = r.get_word_degrees()
        row['rake_key_words'] = list(key_words_dict_scores.keys())        
# Transform key words into bag of words
    df['bag_of_words'] = ''
    for index, row in df.iterrows():
        words = ''
        words += ' '.join(row['rake_key_words']) + ' '
        row['bag_of_words'] = words
    verbose_documentdf = df
    return verbose_documentdf

In [71]:
result = text_to_bagofwords(result)
result

C:\Users\sambe\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\sambe\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,title,category,description,location,rake_key_words,bag_of_words
0,Office Clerk,Admin & Office,"As a Post Office Clerk, you are responsible fo...",US,"[supplies, shipping, also, purchases, work, ha...",supplies shipping also purchases work hands re...
1,Inside Sales Representative,Sales & Retail,This is an inside sales position.\nThis is our...,"Fort Worth, TX","[busiest, time, company, part, successful, env...",busiest time company part successful environme...
2,Entry Level Collections Specialist,Accounting & Finance,Why you should be interested in this role!\nTi...,"Bothell, WA 98021","[debt, supportive, environment, excitement, ex...",debt supportive environment excitement exclusi...
3,Foreign Language Instructor,Education,"As a Foreign Language Instructor for the CIA, ...","Washington, DC","[foreign, language, communications, skills, cr...",foreign language communications skills cross n...
4,QA Tester,Computer & IT,Quality Assurance Testers (“QA Testers”) perfo...,"Baton Rouge, LA 70820","[repetitive, intentions, performance, figure, ...",repetitive intentions performance figure deadl...
...,...,...,...,...,...,...
295,Scor3card Coordinator,Social Services & Nonprofit,The Scor3card Coordinator is will report direc...,"Tulsa, OK","[businesses, responsibility, research, present...",businesses responsibility research presentatio...
296,Data Entry Specialist,Admin & Office,Power Sales Group is a manufacturer's represen...,"Danvers, MA","[enter, heavy, volume, northeastern, us, entry...",enter heavy volume northeastern us entry prepa...
297,Pathway Advisor,Education,This is an exciting time to be an advisor at t...,"Aurora, CO","[students, chosen, guided, academic, pathway, ...",students chosen guided academic pathway debt c...
298,Director Of Food And Beverage,Management,Seeking an experienced and dynamic Director of...,"Sedona, AZ","[3, years, jumped, around, forbes, five, star,...",3 years jumped around forbes five star service...


In [ ]:
all_data = []
final = pd.concat(all_to_add)

In [7]:
def join_and_condense(df):
    # job_descriptions = pd.read_csv('data/job_descriptions.csv', index_col=0)
    with open('recs/pkl/job_descriptions.pkl', 'rb') as f:
        job_descriptions = pickle.load(f)
    job_descriptions = job_descriptions.append(df)
    recommend_df = job_descriptions[['title', 'bag_of_words']]
    recommend_df = recommend_df.reset_index(drop=True)
    return recommend_df

In [8]:
def vectorize_text(df):
    count = CountVectorizer()
    count_matrix = count.fit_transform(df['bag_of_words'])
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    return cosine_sim

In [9]:
def recommend_100(df, matrix):
    recommended_jobs = []
    indices = pd.Series(df['title'])
    idx = indices[indices == 'resume'].index[0]
    score_series = pd.Series(matrix[idx]).sort_values(ascending = False)
    top_100_indices = list(score_series.iloc[1:101].index)

    for i in top_100_indices:
        recommended_jobs.append(list(df['title'])[i])

    return recommended_jobs

In [10]:
def format_recommendations(recommendations):
    jobs10 = []
    for job in recommendations:
        job = job.lower().replace("_", " ").title()
        job = job.replace('Hr Manager', 'HR Manager')
        job = job.replace('Care Giver / Hha / Cna', 'Care Giver')
        jobs10.append(job)
    jobs10 = set(jobs10[0:100])
    format_jobs = list(jobs10)
    # final_jobs10 = jobs10[0:3]
    # for i, item in enumerate(final_jobs10, 1):
        # print(i, '. ' + item + '\n', sep='',end='')
    return format_jobs

In [11]:
def top_100_categories(recommendations):
    # df = pd.read_csv('data/job_descriptions.csv', index_col=0)
    with open('recs/pkl/job_descriptions.pkl', 'rb') as f:
        df = pickle.load(f)
    user_titles = df[df.title.isin(recommendations)]
    user_titles = user_titles[['title', 'category']]
    category_list = list(user_titles.category)
    return category_list

In [12]:
def freq(list_of_categories):
    frequency = []
    unique_words = set(list_of_categories)
    for words in unique_words :
        frequency.append(list_of_categories.count(words))
    return frequency

In [13]:
def viz_data(list_of_categories, frequency_of_categories):
    unique_words = set(list_of_categories)
    unique_words = list(unique_words)
    category_values = dict(zip(unique_words, frequency_of_categories))
    category_dict = {key:val for key, val in category_values.items() if val >= 10}
    names=category_dict.keys()
    size=category_dict.values()
    return names, size

In [14]:
def make_viz(names_of_categories, size_of_categories):
# Create a circle for the center of the plot
    my_circle=plt.Circle( (0,0), 0.7, color='white')
# Give color names
    plt.title('Strength Summary')
    plt.pie(size_of_categories, labels=names_of_categories)
    p=plt.gcf()
    p.gca().add_artist(my_circle)
    plt.show()

In [15]:
def analyze(document_path):
    resume_text = document_to_text(document_path)
    basic_documentdf = compile_document_text(resume_text)
    verbose_documentdf = text_to_bagofwords(basic_documentdf)
    recommend_df = join_and_condense(verbose_documentdf)
    cosine_sim = vectorize_text(recommend_df)
    recommended_jobs = recommend_100(recommend_df, cosine_sim)
    final_jobs10 = format_recommendations(recommended_jobs)
    category_list = top_100_categories(recommended_jobs)
    frequency = freq(category_list)
    names, size = viz_data(category_list, frequency)
    strength_summary = make_viz(names, size)

In [16]:
def final_rec(document_path):
    text = document_to_text(document_path)
    basic_documentdf = compile_document_text(text)
    verbose_documentdf = text_to_bagofwords(basic_documentdf)
    recommend_df = join_and_condense(verbose_documentdf)
    cosine_sim = vectorize_text(recommend_df)
    recommended_jobs = recommend_100(document_path, 'resume', cosine_sim)
    recommendations = format_recommendations(recommended_jobs)
    return recommendations

In [17]:
def upload(document_url):
    resume_text = document_to_text(document_url)
    basic_documentdf = compile_document_text(resume_text)
    verbose_documentdf = text_to_bagofwords(basic_documentdf)
    recommend_df = join_and_condense(verbose_documentdf)
    cosine_sim = vectorize_text(recommend_df)
    recommended_jobs = recommend_100(recommend_df, cosine_sim)
    final_jobs = format_recommendations(recommended_jobs)
    return final_jobs
